# This pynb files contains

1. Linear mixed model (LMM)

2. Bootstrap Analysis (50% cells)

3. Pseudobulk Analysis

# Linear mixed model for identifying differential expression genes

# comparison between low vs int (early) disease group from neocortex

In [ ]:
#Load R packages
library('Matrix')
library('dplyr')
library('ggplot2')
library('MAST')
library("lme4",lib.loc="/home/users/akila20/R/x86_64-pc-linux-gnu-library/4.0/")
library('SingleCellExperiment')
library('zellkonverter')
# Write output 
OUT<-'/scratch/groups/icobos/Akila/recompute/DEG/BA9/unspliced/low_int/'

#Read the h5ad file convert to SCE 

## anndata contains low,int disease group from BA9
example_h5ad <- "/scratch/groups/icobos/Akila/recompute/DEG/BA9/BA9_low_int.h5ad"
sc=readH5AD(example_h5ad)

#### use the raw counts 
counts(sc) <- assay(sc, "unspliced")

#iterate over each celltype and identfy DEG
cluster<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",		"Oligodendrocyte-OPALIN",	"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

cluster1<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",	"Oligodendrocyte-OPALIN",		"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

cluster_count=length(cluster)

## do the mixed model for all the clusters
for (i in 1:cluster_count)
{

sce<-sc[, sc$Author_Annotation == cluster[i]]


logcounts(sce) <- log2(counts(sce) + 1) #log2 transform the raw data**

sca <- SceToSingleCellAssay(sce) # I then upcast the SingleCellExperiment object to MAST’s subclass SingleCellAssay

# cellular detection rate
cdr <-colSums(assay(sca)>0)
colData(sca)$cngeneson <- scale(cdr) # I calculate CDR
scaSample <- sca[sample(which(freq(sca)>.1), 20),]
flat <- as(scaSample, 'data.table')
thres <- thresholdSCRNACountMatrix(assay(sca),cutbins = NULL, nbins = 10, min_per_bin = 30,  bin_by = "median")
assays(sca, withDimnames = FALSE) <- list(thresh=thres$counts_threshold, tpm=assay(sca))

### set the minimum background
freq_expressed <- 0.15
expressed_genes <- freq(sca) > freq_expressed
sca <- sca[expressed_genes,]

#fetch the condition(LOW/INT)

cond <-factor(colData(sca)$Disease.Group)

#set the reference value for condition
cond <-relevel(cond,'int') # In order to have more interpretable coefficients, I set the reference level of my genotype condition to be my wild type cells.
colData(sca)$test <-cond
print(sca@colData$test)


saveRDS(sca,paste0(OUT,"low_int",str(cluster1[i]), "_sca.rds"))

zlm_mixed <- zlm(~test + (1|(subject))  + cngeneson + Age + Sex + total_counts,sca, method = 'glm')
saveRDS(zlm_mixed, paste0(OUT,"low_int",cluster1[i], "_zlm_mixed.rds"))

#calculate logFC between all the fixed effect parameters
logFC <- getLogFC(zlm_mixed)
write.csv(logFC, paste0(OUT,"low_int",cluster1[i],"_logFC_mixed.csv"))

#Log-likelihood model
summary_mixed <- summary(zlm_mixed, doLRT= 'testlow') 
saveRDS(summary_mixed, paste0(OUT,"low_int",cluster1[i],"_summarymixedLRT.rds"))
summarytest_mixed <- summary_mixed$datatable


fcHurdletest_mixed <- merge(summarytest_mixed[contrast=='testlow' & component=='H',.(primerid, `Pr(>Chisq)`)], 
                      summarytest_mixed[contrast=='testlow' & component=='logFC', .(primerid, coef, ci.hi, ci.lo)], 
                      by='primerid')

                      
fcHurdletest_mixed[,fdr:=p.adjust(`Pr(>Chisq)`, 'fdr')]

write.csv(fcHurdletest_mixed, paste0(OUT,"low_int",cluster1[i],"_mixed_Hurdle.csv"))
}

# comparison between int vs high disease group from neocortex

In [ ]:
#Load R packages
library('Matrix')
library('dplyr')
library('ggplot2')
library('MAST')
library("lme4",lib="/home/users/akila20/R/x86_64-pc-linux-gnu-library/4.0/")
library('SingleCellExperiment')
library('zellkonverter')
# Write output 

OUT<-'/scratch/groups/icobos/Akila/recompute/DEG/BA9/unspliced/int_high/'

## anndata contains int,high disease group from BA9
example_h5ad <- "/scratch/groups/icobos/Akila/recompute/DEG/BA9/BA9_int_high.h5ad"
sc=readH5AD(example_h5ad)
counts(sc) <- assay(sc, "unspliced")

#iterate over each celltype and identify DEG

cluster<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",		"Oligodendrocyte-OPALIN",	"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

cluster1<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",	"Oligodendrocyte-OPALIN",		"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

cluster_count=length(cluster)

## do the mixed model for all the clusters
for (i in 1:cluster_count)
{

sce<-sc[, sc$Author_Annotation == cluster[i]]


logcounts(sce) <- log2(counts(sce) + 1) #log2 transform the raw data**

sca <- SceToSingleCellAssay(sce) # I then upcast the SingleCellExperiment object to MAST’s subclass SingleCellAssay

# cellular detection rate
cdr <-colSums(assay(sca)>0)
colData(sca)$cngeneson <- scale(cdr) # I calculate CDR
scaSample <- sca[sample(which(freq(sca)>.1), 20),]
flat <- as(scaSample, 'data.table')
thres <- thresholdSCRNACountMatrix(assay(sca),cutbins = NULL, nbins = 10, min_per_bin = 30,  bin_by = "median")
assays(sca, withDimnames = FALSE) <- list(thresh=thres$counts_threshold, tpm=assay(sca))
    
### set the minimum background
freq_expressed <- 0.15
expressed_genes <- freq(sca) > freq_expressed
sca <- sca[expressed_genes,]


cond <-factor(colData(sca)$Disease.Group)

#set the reference value for condition
cond <-relevel(cond,'high') # In order to have more interpretable coefficients, I set the reference level of my genotype condition to be my wild type cells.
colData(sca)$test <-cond
print(sca@colData$test)


saveRDS(sca,paste0(OUT,"int_high",cluster1[i], "_sca.rds"))

zlm_mixed <- zlm(~test + (1|(subject)) + cngeneson  + Age + Sex  + total_counts,sca, method = 'glm')
saveRDS(zlm_mixed, paste0(OUT,"int_high",cluster1[i], "_zlm_mixed.rds"))

#calculate logFC between all the fixed effect parameters
logFC <- getLogFC(zlm_mixed)
write.csv(logFC, paste0(OUT,"int_high",cluster1[i],"_logFC_mixed.csv"))

#Log-likelihood model
summary_mixed <- summary(zlm_mixed, doLRT= 'testint') 
saveRDS(summary_mixed, paste0(OUT,"int_high",cluster1[i],"_summarymixedLRT.rds"))
summarytest_mixed <- summary_mixed$datatable


fcHurdletest_mixed <- merge(summarytest_mixed[contrast=='testint' & component=='H',.(primerid, `Pr(>Chisq)`)], 
                      summarytest_mixed[contrast=='testint' & component=='logFC', .(primerid, coef, ci.hi, ci.lo)], 
                      by='primerid')

                      
fcHurdletest_mixed[,fdr:=p.adjust(`Pr(>Chisq)`, 'fdr')]

write.csv(fcHurdletest_mixed, paste0(OUT,"int_high",cluster1[i],"_mixed_Hurdle.csv"))
}

# bootstrap LMM comparison between low vs int Disease group (Neocortex)

In [ ]:
#Load R packages
library('Matrix')
library('dplyr')
library('ggplot2')
library('MAST')
library("lme4")
library('SingleCellExperiment')

### specify celltypes for bootstrap

cluster<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",		"Oligodendrocyte-OPALIN",	"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

cluster1<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",	"Oligodendrocyte-OPALIN",		"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

### specify the bootstrap iterations
n_iter=100
cluster_count=length(cluster)
OUT<-"/scratch/groups/icobos/Akila/recompute/bootstrap/unspliced/low_int/"
## do the mixed model for all the clusters
for (j in 1:cluster_count)
{
#### read the cell type specific rds files which contains low,int,high disease group
sc<-readRDS(paste0("/scratch/groups/icobos/Akila/recompute/bootstrap/unspliced/files/",cluster1[j],".rds"))
iter_output <- list()
### bootstrap_iterations
for (i in 1:100)
{
  sce<-sc
  
  ### calculate the 50% cells from low,int disease group
  low <- sce[, sce$Disease.Group == "low"]
  inter<-sce[, sce$Disease.Group == "int"]
  ### number of cells in int/low
  n_int=ncol(inter)
  n_low =ncol(low)
    
  #### fetch 50% cells from low, int disease group for 100 iterations
  frac_to_keep <- 0.5
  keep_low <- floor(n_low*frac_to_keep)
  keep_int <- floor(n_inter*frac_to_keep)
  
  sce_int <- inter[,sample(colnames(inter),keep_int)]
  sce_low <- low[,sample(colnames(low),keep_low)]
  
  
  sce_merged <- cbind(sce_int,sce_low)
  logcounts(sce_merged) <- log2(counts(sce_merged) + 1) #log2 transform the raw data**
  
  sca <- SceToSingleCellAssay(sce_merged) # I then upcast the SingleCellExperiment object to MAST’s subclass SingleCellAssay
  
  # cellular detection rate
  cdr <-colSums(assay(sca)>0)
  colData(sca)$cngeneson <- scale(cdr) # I calculate CDR
  scaSample <- sca[sample(which(freq(sca)>.1), 20),]
  flat <- as(scaSample, 'data.table')
  thres <- thresholdSCRNACountMatrix(assay(sca),cutbins = NULL, nbins = 10, min_per_bin = 30,  bin_by = "median")
  assays(sca, withDimnames = FALSE) <- list(thresh=thres$counts_threshold, tpm=assay(sca))
  freq_expressed <- 0.15
  expressed_genes <- freq(sca) > freq_expressed
  sca <- sca[expressed_genes,]
  
  #fetch the condition(LOW/INT)
  
  cond <-factor(colData(sca)$Disease.Group)
  
  #set the reference value for condition
  cond <-relevel(cond,'int') # In order to have more interpretable coefficients, I set the reference level of my genotype condition to be my wild type cells.
  colData(sca)$test <-cond
  print(sca@colData$test)
  zlm_mixed <- zlm(~test + (1|(subject)) + cngeneson + Age + Sex + total_counts,sca, method = 'glm')

  #calculate logFC between all the fixed effect parameters
  FC <-getLogFC(zlm_mixed)
  filter_FC <- subset(FC, contrast == "testlow")
 
  
  #Log-likelihood model
  summary_mixed <- summary(zlm_mixed, doLRT= 'testlow') 
  summarytest_mixed <- summary_mixed$datatable
  
  
  fcHurdletest_mixed <- merge(summarytest_mixed[contrast=='testlow' & component=='H',.(primerid, `Pr(>Chisq)`)], 
                              summarytest_mixed[contrast=='testlow' & component=='logFC', .(primerid, coef, ci.hi, ci.lo)], 
                              by='primerid')
  
  
  de<-fcHurdletest_mixed[,fdr:=p.adjust(`Pr(>Chisq)`, 'fdr')]
  
  de$logFC <- filter_FC[filter_FC$primerid %in% de$primerid,3]
  iter_output[[i]] <- de
  print(i)
}
### filter FDR <0.05

result <- lapply(iter_output,function(x) x[x$fdr < 0.05,] )

###create matrix for logFC and FDR for the cluster

list_of_genes <- unique(do.call(c,lapply(result,function(x) data.frame(x[[1]])[,1])))
result_matrix_log_fold <- data.frame(matrix(NA,nrow = length(list_of_genes),ncol = n_iter),row.names = list_of_genes)
result_matrix_log_q <- data.frame(matrix(NA,nrow = length(list_of_genes),ncol = n_iter),row.names = list_of_genes )

for(k in 1:length(result)){
  ### get the unique differentially expressed genes from all iterations
  keys_gene <- data.frame(result[[k]])[,1]
  #### fetch logFC and FDR value from each iterations to matrix
  result_matrix_log_fold[keys_gene,k] <- data.frame(result[[k]])[,7]
  result_matrix_log_q[keys_gene,k] <- data.frame(result[[k]])[,6]
}
### Find the mean value of logFC and FDR
result_matrix_log_fold$mean_logFC <- unname(apply(result_matrix_log_fold,1,function(x) mean(x,na.rm = TRUE)))
result_matrix_log_q$mean_qvalue <- unname(apply(result_matrix_log_q,1,function(x) mean(x,na.rm = TRUE)))

write.csv(result_matrix_log_fold,paste0(OUT,"low_int",cluster1[j],"_logFC_iteration.csv"))
write.csv(result_matrix_log_q,paste0(OUT,"low_int",cluster1[j],"qvalue_iteration.csv"))
}

# bootstrap LMM comparison between int vs high Disease group (Neocortex)


In [ ]:
#Load R packages
library('Matrix')
library('dplyr')
library('ggplot2')
library('MAST')
library("lme4")
library('SingleCellExperiment')


### specify celltypes for bootstrap

cluster<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",		"Oligodendrocyte-OPALIN",	"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

cluster1<-c("Astrocyte-GFAP-OSMR",	"Astrocyte-SLC1A2-WIF1",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-GFAP-VCAN",	"OPC",	"Microglia-Reactive-CD163",	"Microglia-Homeostatic-CX3CR1",	"Oligodendrocyte-OPALIN",		"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex16:[FEZF2-SYT6]",	"Ex13:[THEMIS-POSTN]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex14:|FEZF2-HTR2C]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex7:[RORB-GABRG1]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex12:[THEMIS-PRRX1]",	"Ex6:[RORB-MME]",	"Ex11:[RORB-ADGRL4]",	"In2:[LHX6-PVALB-MEPE]",	"In17:[ADARB2-VIP-ABI3BP]",	"In7:[LHX6-SST-B3GAT2]",	"In1:[LHX6-PVALB-CLMN]",	"In3:[LHX6-PVALB-COL15A1]",	"In6:[LHX6-SST-SLC9A2]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In14:[ADARB2-SEMA3C]",	"In19:[ADARB2-CALB2-SCML4]",	"In18:[ADARB2-VIP-EXPH5]",	"In11:[ADARB2-LAMP5-KIT]",	"In4:[LHX6-SST-MSR1]",	"In9:[LHX6-SST-NPY]",	"In5:[LHX6-SST-SPON1]",	"In8:[LHX6-SST-SEL1L3]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]")

cluster_count=length(cluster)
OUT<-"/scratch/groups/icobos/Akila/recompute/bootstrap/unspliced/int_high/"
## do the mixed model for all the clusters
for (j in 1:cluster_count)
{
#### read the cell type specific rds files which contains low,int,high disease group

sc<-readRDS(paste0("/scratch/groups/icobos/Akila/recompute/bootstrap/unspliced/files/",cluster1[j],".rds"))
iter_output <- list()
    
### specify the bootstrap iterations
for (i in 1:100)
{
  sce<-sc
  
######calculate the 50% cells from int, high disease group
    
  inter <- sce[, sce$Disease.Group == "int"]
  high<-sce[, sce$Disease.Group == "high"]
 
  ### number of cells in int/high
    
  n_int=ncol(inter)
  n_high=ncol(high)
    
  #### fetch 50% cells from int, high disease group for 100 iterations

  frac_to_keep <- 0.5
  keep_int <- floor(n_int*frac_to_keep)
  keep_high <- floor(n_high*frac_to_keep)
  
  sce_int <- inter[,sample(colnames(inter),keep_int)]
  sce_high <- high[,sample(colnames(high),keep_high)]
  
  
  sce_merged <- cbind(sce_int,sce_high)
  logcounts(sce_merged) <- log2(counts(sce_merged) + 1) #log2 transform the raw data**
  
  sca <- SceToSingleCellAssay(sce_merged) # I then upcast the SingleCellExperiment object to MAST’s subclass SingleCellAssay
  
  # cellular detection rate
  cdr <-colSums(assay(sca)>0)
  colData(sca)$cngeneson <- scale(cdr) # I calculate CDR
  scaSample <- sca[sample(which(freq(sca)>.1), 20),]
  flat <- as(scaSample, 'data.table')
  thres <- thresholdSCRNACountMatrix(assay(sca),cutbins = NULL, nbins = 10, min_per_bin = 30,  bin_by = "median")
  assays(sca, withDimnames = FALSE) <- list(thresh=thres$counts_threshold, tpm=assay(sca))
  freq_expressed <- 0.15
  expressed_genes <- freq(sca) > freq_expressed
  sca <- sca[expressed_genes,]
  
  #fetch the condition(INT/HIGH)
  
  cond <-factor(colData(sca)$Disease.Group)
  
  #set the reference value for condition
  cond <-relevel(cond,'high') # In order to have more interpretable coefficients, I set the reference level of my genotype condition to be my wild type cells.
  colData(sca)$test <-cond
  
  zlm_mixed <- zlm(~test + (1|(subject)) + cngeneson + Age + Sex + RIN + total_counts,sca, method = 'glm')
 

  #calculate logFC between all the fixed effect parameters
  FC <-getLogFC(zlm_mixed)
  filter_FC <- subset(FC, contrast == "testint")
    
   
  #Log-likelihood model
  summary_mixed <- summary(zlm_mixed, doLRT= 'testint') 
  summarytest_mixed <- summary_mixed$datatable
  
  
  fcHurdletest_mixed <- merge(summarytest_mixed[contrast=='testint' & component=='H',.(primerid, `Pr(>Chisq)`)], 
                              summarytest_mixed[contrast=='testint' & component=='logFC', .(primerid, coef, ci.hi, ci.lo)], 
                              by='primerid')
  
  
  de<-fcHurdletest_mixed[,fdr:=p.adjust(`Pr(>Chisq)`, 'fdr')]
  
    de$logFC <- filter_FC[filter_FC$primerid %in% de$primerid,3]
  iter_output[[i]] <- de
  print(i)
  
}
### filter FDR <0.05

result <- lapply(iter_output,function(x) x[x$fdr < 0.05,] )

###create matrix for logFC and FDR for the cluster
n_iter=100
list_of_genes <- unique(do.call(c,lapply(result,function(x) data.frame(x[[1]])[,1])))
result_matrix_log_fold <- data.frame(matrix(NA,nrow = length(list_of_genes),ncol = n_iter),row.names = list_of_genes)
result_matrix_log_q <- data.frame(matrix(NA,nrow = length(list_of_genes),ncol = n_iter),row.names = list_of_genes )
print(cluster1[j])
for(k in 1:length(result)){
  ### get the unique differentially expressed genes from all iterations
  keys_gene <- data.frame(result[[k]])[,1]
  #### fetch logFC and FDR value from each iterations to matrix
  result_matrix_log_fold[keys_gene,k] <- data.frame(result[[k]])[,7]
  result_matrix_log_q[keys_gene,k] <- data.frame(result[[k]])[,6]
}
### Find the mean value of logFC and FDR
result_matrix_log_fold$mean_logFC <- unname(apply(result_matrix_log_fold,1,function(x) mean(x,na.rm = TRUE)))
result_matrix_log_q$mean_qvalue <- unname(apply(result_matrix_log_q,1,function(x) mean(x,na.rm = TRUE)))

write.csv(result_matrix_log_fold,paste0(OUT,"int_high",cluster1[j],"_logFC_iteration.csv"))
write.csv(result_matrix_log_q,paste0(OUT,"int_high",cluster1[j],"qvalue_iteration.csv"))
}

# pseudobulk comparison between (low vs Int-> Disease Group) (Neocortex)

In [ ]:
import scanpy as sc
import pandas as pd
import random
import numpy as np
import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

##load data
bdata=sc.read_h5ad("/scratch/groups/icobos/Akila/recompute/QC_test/dataset.h5ad")

#### selet BA9
bdata=bdata[bdata.obs["Brain.Region"].isin(["Frontal Cx (BA9)"])]

##select Early pathology
early=bdata[bdata.obs["Disease.Group"].isin(["low","int"])]

##define celltypes
list1=["Astrocyte-GFAP-OSMR",	"Astrocyte-GFAP-VCAN",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-SLC1A2-WIF1",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex6:[RORB-MME]",	"Ex7:[RORB-GABRG1]",	"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex11:[RORB-ADGRL4]",	"Ex12:[THEMIS-PRRX1]",	"Ex13:[THEMIS-POSTN]",	"Ex14:|FEZF2-HTR2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex16:[FEZF2-SYT6]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"In1:[LHX6-PVALB-CLMN]",	"In2:[LHX6-PVALB-MEPE]",	"In3:[LHX6-PVALB-COL15A1]",	"In4:[LHX6-SST-MSR1]",	"In5:[LHX6-SST-SPON1]",	"In6:[LHX6-SST-SLC9A2]",	"In7:[LHX6-SST-B3GAT2]",	"In8:[LHX6-SST-SEL1L3]",	"In9:[LHX6-SST-NPY]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]",	"In11:[ADARB2-LAMP5-KIT]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In14:[ADARB2-SEMA3C]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In17:[ADARB2-VIP-ABI3BP]",	"In18:[ADARB2-VIP-EXPH5]",	"In19:[ADARB2-CALB2-SCML4]",	"Microglia-Homeostatic-CX3CR1",	"Microglia-Reactive-AIF1",	"Microglia-Reactive-CACNA1B",	"Microglia-Reactive-CD163",	"OPC",	"Oligodendrocyte-COL18A1",	"Oligodendrocyte-OPALIN",	"Pericytes",	"VLMC",	"endothelial",]

for i in range(len(list1)):
    ##subset each celltype
    cell_subset=early[early.obs["Author_Annotation"].isin([list1[i]])]
    ##create peudobulk aggregate based on each subject
    pbs=[]
    for sample in cell_subset.obs["subject"].unique():
        samp_cell_subset=cell_subset[cell_subset.obs["subject"] == sample]
        samp_cell_subset.X=samp_cell_subset.layers["unspliced"]
        rep_adata =sc.AnnData(X=samp_cell_subset.X.sum(axis=0),var=samp_cell_subset.var[[]])
        rep_adata.obs_names=[sample]
        rep_adata.obs["Disease.Group"]=samp_cell_subset.obs["Disease.Group"].iloc[0]
        pbs.append(rep_adata)
        
    ## create counts for DDS
    pb=sc.concat(pbs)
    import pydeseq2
    from pydeseq2.utils import wald_test
    from pydeseq2.dds import DeseqDataSet
    from pydeseq2.ds import DeseqStats
    counts=pd.DataFrame(pb.X,columns=pb.var_names)
    dds=DeseqDataSet(counts=counts,clinical=pb.obs,design_factors= "Disease.Group")
    sc.pp.normalize_total(dds)
    sc.pp.log1p(dds)
    sc.pp.filter_genes(dds,min_cells=10)
    from pydeseq2.utils import test_valid_counts
    test_valid_counts(counts)
    dds=DeseqDataSet(counts=counts,clinical=pb.obs,design_factors= "Disease.Group")
    sc.pp.filter_genes(dds,min_cells=10)
    dds.deseq2()
    sc.tl.pca(dds)
    stat_res = DeseqStats(dds, contrast=("Disease.Group", "int", "low"),alpha=0.05)
    stat_res.summary()
    de = stat_res.results_df
    de.sort_values('stat', ascending=False)
    de.to_csv("early/unspliced/BA9/"+str(list1[i])+"_pseudobulk.csv")


# pseudobulk comparison between (Int vs High-> Disease Group) (Neocortex)

In [ ]:
import scanpy as sc
import pandas as pd
import random
import numpy as np
import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

##load data
bdata=sc.read_h5ad("/scratch/groups/icobos/Akila/recompute/QC_test/dataset.h5ad")

#### selet BA9
bdata=bdata[bdata.obs["Brain.Region"].isin(["Frontal Cx (BA9)"])]

#### select disease group
late=bdata[bdata.obs["Disease.Group"].isin(["int","high"])]
##define celltypes
list1=["Astrocyte-GFAP-OSMR",	"Astrocyte-GFAP-VCAN",	"Astrocyte-SLC1A2-SMTN",	"Astrocyte-SLC1A2-WIF1",	"Ex1:[CUX2-SERPINE2-LAMP5]",	"Ex2:[CUX2-RORB-GLIS3-LRRC2]",	"Ex3:[CUX2-RORB-GLIS3-SV2C]",	"Ex4:[CUX2-RORB-COL5A2-CLMN]",	"Ex5:[RORB-CUX2-EYA4]",	"Ex6:[RORB-MME]",	"Ex7:[RORB-GABRG1]",	"Ex8:[RORB-TLL1-TMTC4]",	"Ex9:[RORB-TLL1-PCP4-MID1]",	"Ex10:[RORB-TLL1-PCP4-TYR]",	"Ex11:[RORB-ADGRL4]",	"Ex12:[THEMIS-PRRX1]",	"Ex13:[THEMIS-POSTN]",	"Ex14:|FEZF2-HTR2C]",	"Ex15:[FEZF2-ADRA1A]",	"Ex16:[FEZF2-SYT6]",	"Ex17:[FEZF2-ZFHX3-SEMA3D]",	"Ex18:[FEZF2-ZFHX3-SCUBE1]",	"In1:[LHX6-PVALB-CLMN]",	"In2:[LHX6-PVALB-MEPE]",	"In3:[LHX6-PVALB-COL15A1]",	"In4:[LHX6-SST-MSR1]",	"In5:[LHX6-SST-SPON1]",	"In6:[LHX6-SST-SLC9A2]",	"In7:[LHX6-SST-B3GAT2]",	"In8:[LHX6-SST-SEL1L3]",	"In9:[LHX6-SST-NPY]",	"In10:[LHX6-ADARB2-LAMP5-HCRTR2]",	"In11:[ADARB2-LAMP5-KIT]",	"In12:[ADARB2-LAMP5-NDNF]",	"In13:[ADARB2-SYT6]",	"In14:[ADARB2-SEMA3C]",	"In15:[ADARB2-VIP-DACH2]",	"In16:[ADARB2-VIP-TAC3-NRP1]",	"In17:[ADARB2-VIP-ABI3BP]",	"In18:[ADARB2-VIP-EXPH5]",	"In19:[ADARB2-CALB2-SCML4]",	"Microglia-Homeostatic-CX3CR1",	"Microglia-Reactive-AIF1",	"Microglia-Reactive-CACNA1B",	"Microglia-Reactive-CD163",	"OPC",	"Oligodendrocyte-COL18A1",	"Oligodendrocyte-OPALIN",	"Pericytes",	"VLMC",	"endothelial",]

for i in range(len(list1)):
    ##subset each celltype
    cell_subset=late[late.obs["author_annotation"].isin([list1[i]])]
    ##create peudobulk aggregate based on each subject
    pbs=[]
    for sample in cell_subset.obs["subject"].unique():
        samp_cell_subset=cell_subset[cell_subset.obs["subject"] == sample]
        samp_cell_subset.X=samp_cell_subset.layers["unspliced"]
        rep_adata =sc.AnnData(X=samp_cell_subset.X.sum(axis=0),var=samp_cell_subset.var[[]])
        rep_adata.obs_names=[sample]
        rep_adata.obs["Disease.Group"]=samp_cell_subset.obs["Disease.Group"].iloc[0]
        pbs.append(rep_adata)
    ## create counts for DDS
    pb=sc.concat(pbs)
    import pydeseq2
    from pydeseq2.utils import wald_test
    from pydeseq2.dds import DeseqDataSet
    from pydeseq2.ds import DeseqStats
    counts=pd.DataFrame(pb.X,columns=pb.var_names)
    dds=DeseqDataSet(counts=counts,clinical=pb.obs,design_factors= "Disease.Group")
    sc.pp.normalize_total(dds)
    sc.pp.log1p(dds)
    sc.pp.filter_genes(dds,min_cells=10)
    from pydeseq2.utils import test_valid_counts
    test_valid_counts(counts)
    dds=DeseqDataSet(counts=counts,clinical=pb.obs,design_factors= "Disease.Group")
    sc.pp.filter_genes(dds,min_cells=10)
    dds.deseq2()
    sc.tl.pca(dds)
    stat_res = DeseqStats(dds, contrast=("Disease.Group", "high", "int"),alpha=0.05)
    stat_res.summary()
    de = stat_res.results_df
    de.sort_values('stat', ascending=False)
    de.to_csv("late/unspliced/BA9/"+str(list1[i])+"_pseudobulk.csv")
